In [1]:
#installing all dependencies
!pip install torchattacks
!pip install torch==1.7.1 torchvision==0.8.2 
!pip install torch-dct

^C
  Using cached torch-1.7.1-cp38-cp38-win_amd64.whl (184.0 MB)

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip


ERROR: Could not find a version that satisfies the requirement torchvision==0.8.2 (from versions: 0.1.6, 0.1.7, 0.1.8, 0.1.9, 0.2.0, 0.2.1, 0.2.2, 0.2.2.post2, 0.2.2.post3, 0.5.0, 0.9.0, 0.9.1, 0.10.0, 0.10.1, 0.11.0, 0.11.1, 0.11.2, 0.11.3, 0.12.0, 0.13.0, 0.13.1)
ERROR: No matching distribution found for torchvision==0.8.2



[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip
^C


In [2]:
#Try updated form
#!pip install torchattacks
#!pip install torch
#!pip install torch-dct

In [3]:
#importing all necessary libraries
#import torch_dct as dct
import matplotlib.pyplot as plt
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision.utils
from torchvision import models
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch_dct as dct

import torchattacks
from torchattacks import PGD, FGSM, FFGSM
#import dct_upd
from slide import SLIDE
from ffgsm_dct import FFGSM_DCT
from pgd_dct import PGD_DCT
#from pgd_fft import PGD_FFT

from models import CNN
import random
from stochastic_attack import STOCHASTIC_ATTACK


[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip


C:\Users\Diego\anaconda3\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: [WinError 127] The specified procedure could not be found
  warn(f"Failed to load image Python extension: {e}")


In [4]:
#Downloading Fashion MNIST Dataset
train_list_transforms = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        #transforms.Normalize((0.1307,), (0.3081,)),
    ])

fashion_mnist_train = dsets.FashionMNIST(root='./data1/',
                          train=True,
                          transform=train_list_transforms,
                          download=True)

fashion_mnist_aux = dsets.FashionMNIST(root='./data1/',
                         train=False,
                         transform=train_list_transforms,
                         download=True)

fashion_mnist_test, fashion_mnist_valid = torch.utils.data.random_split(fashion_mnist_aux, [5000, 5000], generator=torch.Generator().manual_seed(42))

In [5]:
batch_size = 128
torch.manual_seed(0)

train_loader  = torch.utils.data.DataLoader(dataset=fashion_mnist_train,
                                           batch_size=batch_size,
                                           shuffle=42,num_workers=3,worker_init_fn= lambda id : np.random.seed(id))

test_loader = torch.utils.data.DataLoader(dataset=fashion_mnist_test,
                                         batch_size=batch_size,
                                         shuffle=42,num_workers=3,worker_init_fn= lambda id : np.random.seed(id))

valid_loader = torch.utils.data.DataLoader(dataset=fashion_mnist_valid,
                                         batch_size=batch_size,
                                         shuffle=42,num_workers=3,worker_init_fn= lambda id : np.random.seed(id))

In [6]:
#Accuracy table
df_acc = pd.DataFrame({'Model' : [], 'STD' : [], 'Rand': [], 'FGSM' : [], 'FFGSM' : [], 'SLIDE' : [], 'PGD' : [], 'FFGSM_DCT' : [], 'PGD_DCT' : []})

In [7]:
#defining model and parameters
model = CNN().cuda()
model_round_rob = CNN().cuda()
model_greedy = CNN().cuda()
model_mult_prob = CNN().cuda()
model_rand = CNN().cuda()
model_pgd = CNN().cuda()
model_pgd_dct = CNN().cuda()
model_ffgsm = CNN().cuda()
model_ffgsm_dct = CNN().cuda()

optimizer = optim.Adam(model.parameters(), lr=0.001)
optimizer_round_rob = optim.Adam(model_round_rob.parameters(), lr=0.001)
optimizer_greedy = optim.Adam(model_greedy.parameters(), lr=0.001)
optimizer_mult_prob = optim.Adam(model_mult_prob.parameters(), lr=0.001)
optimizer_rand = optim.Adam(model_rand.parameters(), lr=0.001)
optimizer_pgd = optim.Adam(model_pgd.parameters(), lr=0.001)
optimizer_pgd_dct = optim.Adam(model_pgd_dct.parameters(), lr=0.001)
optimizer_ffgsm = optim.Adam(model_ffgsm.parameters(), lr=0.001)
optimizer_ffgsm_dct = optim.Adam(model_ffgsm_dct.parameters(), lr=0.001)

loss = nn.CrossEntropyLoss()
num_epochs = 10

model

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
#modified test function with 3 attacks and toggle between Pixel and DCT representation.Default is Pixel

def get_accuracy(model, test_loader, attack=None):
    model.eval()

    correct = 0
    total = 0
    
    if (attack==None):
      atk = lambda x, y: x
    elif (attack=="RANDOM") :
      atk = STOCHASTIC_ATTACK(model, eps=0.3)
    elif (attack=="FGSM") :
      atk = FGSM(model, eps=0.3)
    elif (attack=="FFGSM") :
      atk = FFGSM(model, eps=0.3)
    elif (attack=="SLIDE") :
      atk = SLIDE(model, eps=0.3)
    elif (attack=="PGD") :
      atk = PGD(model, eps=0.3)
    elif (attack=="FFGSM_DCT") :
      atk = FFGSM_DCT(model, eps=0.3)
    elif (attack=="PGD_DCT") :
      atk = PGD_DCT(model, eps=0.3)
    

    for images, labels in test_loader:

        images = atk(images, labels).cuda()              
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels.cuda()).sum()

    acc = 100 * float(correct) / total
    #print(f"Attack: {attack}, Accuracy: {acc}")

    return acc

In [ ]:
#train function with PGD attack and toggle between DCT and Pixel representation .Default is Pixel

def train_(model, train_loader, optimizer, loss, attack=None, num_epochs=5):

    model.train()
    
    if (attack==None):
      atk = lambda x, y: x
    elif (attack=="RANDOM") :
      atk = STOCHASTIC_ATTACK(model, eps=0.3)
    elif (attack=="FGSM") :
      atk = FGSM(model, eps=0.3)
    elif (attack=="FFGSM") :
      atk = FFGSM(model, eps=0.3)
    elif (attack=="SLIDE") :
      atk = SLIDE(model, eps=0.3)
    elif (attack=="PGD") :
      atk = PGD(model, eps=0.3)
    elif (attack=="FFGSM_DCT") :
      atk = FFGSM_DCT(model, eps=0.3)
    elif (attack=="PGD_DCT") :
      atk = PGD_DCT(model, eps=0.3)

    for epoch in range(num_epochs):

        total_batch = len(fashion_mnist_train) // batch_size
        
        for i, (batch_images, batch_labels) in enumerate(train_loader):

           
            Y = batch_labels.cuda()
            X = atk(batch_images, batch_labels).cuda()

            pre = model(X)
            cost = loss(pre, Y)

            optimizer.zero_grad()
            cost.backward()
            optimizer.step()

            if (i+1) % 100 == 0:
                print(f'Atk: {attack}, Epoch [{epoch+1}/{num_epochs}], lter [{i+1}/{total_batch}], Loss: {cost.item()}')
    return cost

In [ ]:
def round_robin(model, train_loader, optimizer, loss, num_epochs=5):

    model.train()
    
    atks_list = []
    atks_list.append(lambda x, y: x)
    atks_list.append(STOCHASTIC_ATTACK(model, eps=0.3))
    atks_list.append(FGSM(model, eps=0.3))
    atks_list.append(FFGSM(model, eps=0.3))
    atks_list.append(SLIDE(model, eps=0.3))
    atks_list.append(PGD(model, eps=0.3))
    atks_list.append(FFGSM_DCT(model, eps=0.3))
    atks_list.append(PGD_DCT(model, eps=0.3))

    num_attacks = len(atks_list)
    total_batch = len(fashion_mnist_train) // batch_size

    for epoch in range(num_epochs): #+1 to round up

      count = 0

      for i, (batch_images, batch_labels) in enumerate(valid_loader):

          atk = atks_list[count]
          Y = batch_labels.cuda()
          X = atk(batch_images, batch_labels).cuda()

          pre = model(X)
          cost = loss(pre, Y)

          optimizer.zero_grad()
          cost.backward()
          optimizer.step()

          count += 1
          count %= num_attacks

          if (i+1) % 100 == 0:
              print(f'Epoch [{epoch+1}/{num_epochs}], lter [{i+1}/{total_batch}], Loss: {cost.item()}')
    return cost

In [ ]:
def greedy(model, train_loader, valid_loader, optimizer, loss, num_epochs=5):

    
    atks_list = []
    atks_list.append(lambda x, y: x)
    atks_list.append(STOCHASTIC_ATTACK(model, eps=0.3))
    atks_list.append(FGSM(model, eps=0.3))
    atks_list.append(FFGSM(model, eps=0.3))
    atks_list.append(SLIDE(model, eps=0.3))
    atks_list.append(PGD(model, eps=0.3))
    atks_list.append(FFGSM_DCT(model, eps=0.3))
    atks_list.append(PGD_DCT(model, eps=0.3))

    num_attacks = len(atks_list)
    total_batch = len(fashion_mnist_train) // batch_size

    for epoch in range(num_epochs):

      loss_list = [0]*num_attacks

      model.eval()

      for i, (batch_images, batch_labels) in enumerate(valid_loader):

          Y = batch_labels.cuda()
          current_batch_size = Y.shape[0]

          for i, atk in enumerate(atks_list):
          
              X = atk(batch_images, batch_labels).cuda()
              
              pre = model(X)
              cost = loss(pre, Y)

              loss_list[i] += float(cost)/current_batch_size

      max_loss_arg = np.argmax(np.array(loss_list))
      atk = atks_list[max_loss_arg]

      print(f"Epoch [{epoch+1}/{num_epochs}], Worst loss attack: {atk.attack}, loss: {loss_list[max_loss_arg]}")

      model.train()

      for i, (batch_images, batch_labels) in enumerate(train_loader):

          Y = batch_labels.cuda()
          X = atk(batch_images, batch_labels).cuda()

          pre = model(X)
          cost = loss(pre, Y)

          optimizer.zero_grad()
          cost.backward()
          optimizer.step()

          if (i+1) % 100 == 0:
              print(f'Epoch [{epoch+1}/{num_epochs}], lter [{i+1}/{total_batch}], Loss: {cost.item()}')
    return cost

In [ ]:
def multivariate_probability(model, train_loader, valid_loader, optimizer, loss, num_epochs=5):

    atks_list = []
    atks_list.append(lambda x, y: x)
    atks_list.append(STOCHASTIC_ATTACK(model, eps=0.3))
    atks_list.append(FGSM(model, eps=0.3))
    atks_list.append(FFGSM(model, eps=0.3))
    atks_list.append(SLIDE(model, eps=0.3))
    atks_list.append(PGD(model, eps=0.3))
    atks_list.append(FFGSM_DCT(model, eps=0.3))
    atks_list.append(PGD_DCT(model, eps=0.3))

    num_attacks = len(atks_list)
    total_batch = len(fashion_mnist_train) // batch_size
    w_list = [1]*num_attacks
    eta = 0.1

    for epoch in range(num_epochs):

      loss_list = [0]*num_attacks

      model.eval()

      for i, (batch_images, batch_labels) in enumerate(valid_loader):

          Y = batch_labels.cuda()
          current_batch_size = Y.shape[0]

          for i, atk in enumerate(atks_list):
          
              X = atk(batch_images, batch_labels).cuda()
              
              pre = model(X)
              cost = loss(pre, Y)

              loss_list[i] += float(cost)/current_batch_size

      loss_list = np.array(loss_list)
      total_loss = loss_list.sum()
      prob_list = np.cumsum(loss_list)
      prob_list /= total_loss

      print(f"Epoch [{epoch+1}/{num_epochs}], atks: (None, Rand, FGSM, FFGSM, SLIDE, PGD, FFGSM_DCT, PGD_DCT), prob list: {loss_list}")

      model.train()

      for i, (batch_images, batch_labels) in enumerate(train_loader):

          rand = random.random()
          atk_idx = 0
          for j in range(1,num_attacks):
              if rand > prob_list[j]:
                  atk_idx = j
              else:
                  break

          atk = atks_list[atk_idx]
          Y = batch_labels.cuda()
          X = atk(batch_images, batch_labels).cuda()

          pre = model(X)
          cost = loss(pre, Y)

          optimizer.zero_grad()
          cost.backward()
          optimizer.step()

          if (i+1) % 100 == 0:
              print(f'Epoch [{epoch+1}/{num_epochs}], lter [{i+1}/{total_batch}], Loss: {cost.item()}')
    return cost

In [ ]:
def get_and_add_acc_model(model, test_loader, df, model_name="Model"):
    model_std_acc = get_accuracy(model, test_loader)
    model_rand_acc = get_accuracy(model, test_loader, "RANDOM")
    model_fgsm_acc = get_accuracy(model, test_loader, "FGSM")
    model_ffgsm_acc = get_accuracy(model, test_loader, "FFGSM")
    model_slide_acc = get_accuracy(model, test_loader, "SLIDE")
    model_pgd_acc = get_accuracy(model, test_loader, "PGD")
    model_ffgsm_dct_acc = get_accuracy(model, test_loader, "FFGSM_DCT")
    model_pgd_dct_acc = get_accuracy(model, test_loader, "PGD_DCT")
    model_acc = pd.Series([model_name, model_std_acc, model_rand_acc, model_fgsm_acc, model_ffgsm_acc, model_slide_acc, model_pgd_acc, model_ffgsm_dct_acc, model_pgd_dct_acc], index=df_acc.columns)
    df = df.append(model_acc, ignore_index = True)
    return df

In [ ]:
cost0=train_(model,train_loader,optimizer,loss)

In [ ]:
df_acc = get_and_add_acc_model(model, test_loader, df_acc, model_name="Std")
df_acc

In [ ]:
cost_round0 = train_(model_round_rob,train_loader,optimizer_round_rob,loss)
cost_round1 = round_robin(model_round_rob,train_loader,optimizer_round_rob,loss)

In [ ]:
df_acc = get_and_add_acc_model(model_round_rob, test_loader, df_acc, model_name="RR")
df_acc

In [ ]:
cost_greedy0 = train_(model_greedy,train_loader,optimizer_greedy,loss)
cost_greedy1 = greedy(model_greedy,train_loader,valid_loader,optimizer_greedy,loss)

In [ ]:
df_acc = get_and_add_acc_model(model_greedy, test_loader, df_acc, model_name="Greedy")
df_acc

In [ ]:
cost_mult_prob0 = train_(model_mult_prob,train_loader,optimizer_mult_prob,loss)
cost_mult_prob1 = multivariate_probability(model_mult_prob,train_loader,valid_loader,optimizer_mult_prob,loss)

In [ ]:
df_acc = get_and_add_acc_model(model_mult_prob, test_loader, df_acc, model_name="MP")
df_acc

In [ ]:
#Initial training in Pixel representation
cost_rand0 = train_(model_rand, train_loader, optimizer_rand, loss)
cost_rand1 = train_(model_rand, train_loader, optimizer_rand, loss, "RANDOM")

In [ ]:
df_acc = get_and_add_acc_model(model_rand, test_loader, df_acc, "Rand")
df_acc

In [ ]:
#Initial training in Pixel representation
cost_pgd0 = train_(model_pgd, train_loader, optimizer_pgd, loss)
cost_pgd1 = train_(model_pgd, train_loader, optimizer_pgd, loss, "PGD")

In [ ]:
df_acc = get_and_add_acc_model(model_pgd, test_loader, df_acc, model_name="Pgd")
df_acc

In [ ]:
#Initial training in Pixel representation
cost_pgd_dct0 = train_(model_pgd_dct, train_loader, optimizer_pgd_dct, loss)
cost_pgd_dct1 = train_(model_pgd_dct, train_loader, optimizer_pgd_dct, loss, "PGD_DCT")

In [ ]:
df_acc = get_and_add_acc_model(model_pgd_dct, test_loader, df_acc, model_name="Pgd_dct")
df_acc

In [ ]:
#Loss when converting from Pixel to DCT

examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)
model_used = model_mult_prob

atks_names = ['None', 'Rand', 'FGSM', 'FFGSM', 'SLIDE', 'PGD', 'FFGSM_DCT', 'PGD_DCT']
atks_list = []
atks_list.append(lambda x, y: x)
atks_list.append(STOCHASTIC_ATTACK(model_used, eps=0.3))
atks_list.append(FGSM(model_used, eps=0.3))
atks_list.append(FFGSM(model_used, eps=0.3))
atks_list.append(SLIDE(model_used, eps=0.3))
atks_list.append(PGD(model_used, eps=0.3))
atks_list.append(FFGSM_DCT(model_used, eps=0.3))
atks_list.append(PGD_DCT(model_used, eps=0.3))

In [ ]:
#Pixel representation
import matplotlib.pyplot as plt

num_images = 5
num_attacks = len(atks_list)
img_size = 3
fig, axs = plt.subplots(num_attacks, num_images, constrained_layout=True, figsize=(3*num_attacks, 24))

for i in range(num_attacks):
    for j in range(num_images):
        image = example_data[j:j+1,:1,:,:]
        label = example_targets[j:j+1]
        atk = atks_list[i]
        images_atk = atk(image, label).cuda()
        pred = model_used(images_atk)
        _, predicted = torch.max(pred.data, 1)
        correct = (predicted == label.cuda())
        #avg = np.average(images_atk.cpu().numpy())
        axs[i,j].imshow(images_atk.cpu().numpy()[0,0], cmap='gray', interpolation='none')
        axs[i,j].set_title(f'{atks_names[i]}, correct: {correct[0]}') #avg: {avg}





In [ ]:
#Pixel representation
import matplotlib.pyplot as plt

num_images = 5
num_attacks = len(atks_list)
img_size = 3
fig, axs = plt.subplots(num_attacks, num_images, constrained_layout=True, figsize=(3*num_attacks, 24))

for i in range(num_attacks):
    for j in range(num_images):
        image = example_data[j:j+1,:1,:,:]
        label = example_targets[j:j+1]
        atk = atks_list[i]
        images_atk = atk(image, label).cuda()
        pred = model_used(images_atk)
        _, predicted = torch.max(pred.data, 1)
        correct = (predicted == label.cuda())
        #avg = np.average(images_atk.cpu().numpy())
        axs[i,j].imshow(images_atk.cpu().numpy()[0,0], cmap='gray', interpolation='none')
        axs[i,j].set_title(f'{atks_names[i]}, correct: {correct[0]}') #avg: {avg}





In [ ]:
#DCT representation
num_images = 128
img_size = 3
#fig, axs = plt.subplots(2, num_images, constrained_layout=True, figsize=(20,8))#img_size*2, img_size*num_images))
max = -1000000000000000000000000
min = 10000000000000000000000000

for j in range(num_images):
    image = example_data[j:j+1,:1,:,:]
    img_max = np.amax(image.numpy())
    img_min = np.amin(image.numpy())
    if img_max > max:
        max = img_max
    if img_min < min:
        min = img_min
    """
    for i in range(2):
        title = 'Normal'
        if i%2 == 1:
            image = dct.idct(dct.dct(image))
            title = 'Dct conversion'
        axs[i,j].imshow(image[0,0], cmap='gray', interpolation='none')
        axs[i,j].set_title(f'{title}')
    """

print(min, max)

In [ ]:
#DCT representation
num_images = 128
img_size = 3
#fig, axs = plt.subplots(2, num_images, constrained_layout=True, figsize=(20,8))#img_size*2, img_size*num_images))
max = -1000000000000000000000000
min = 10000000000000000000000000

for j in range(num_images):
    image = example_data[j:j+1,:1,:,:]
    img_max = np.amax(image.numpy())
    img_min = np.amin(image.numpy())
    if img_max > max:
        max = img_max
    if img_min < min:
        min = img_min
    """
    for i in range(2):
        title = 'Normal'
        if i%2 == 1:
            image = dct.idct(dct.dct(image))
            title = 'Dct conversion'
        axs[i,j].imshow(image[0,0], cmap='gray', interpolation='none')
        axs[i,j].set_title(f'{title}')
    """

print(min, max)

In [ ]:
#MSE LOSS between the two tensors
input = example_data
target = dct.idct(dct.dct(example_data))
mse_loss = nn.MSELoss()
output = mse_loss(input, target)


#print('input: ', input)
#print('target: ', target)
print('output: ', output)